# Blinky

This blinky variant simply toggles a LED when a counter unit reaches  certain value. First, we import the necessary cyHDL API:

In [1]:
from cyhdl import *

We define a basic type `Bool` and a `Register` type of with 8:

In [2]:
Bool = Signal.Type(bool)
Register = Signal.Type(intbv, 10)

Finally, we define a hardware block with a few pins and a `cfg` register:

In [3]:
@block
def blinky(clk   : ClkSignal,
           en    : Bool,
           reset : ResetSignal,
           outp  : Bool.Output,
           cfg   : Register  ):

    counter = Signal(intbv(0)[16:])
    val = Bool(0)
    
    @always_seq(clk.posedge, reset)
    def worker():
        # When enable set, set counter's .next value to incremented
            
        if counter[:8] == cfg[8:]:
            val.next = ~val # remember: assign signals using .next! 
            counter.next = 0
        elif en:
            counter.next = counter + 1

            
    # This is a simple list of primitives. A .wireup is a simple assignment outside
    # a process
    logic = [
        outp.wireup(val)
            ]
    
    return instances()

## Testing/simulating the unit

In order to simulate the functionality of the blinky unit, we need to build a test bench - like virtual peripherals stimulating and probing the pins of the unit under test (`uut`).

We use an external simulator (GHDL or Icarus Verilog) in this configuration. The testbench including the uut is emit to the target HDL and passed on to the chosen simulator backend. There is no co-simulation taking place in this scenario.

Because the testbench code is transpiled to HDL, a `@block` decorator is present. Note that a testbench executed in native Python (using a different `Sim` simulator backend) would omit that decorator.

Because Signal classes can depend on the simulator, future compatible practise is to use the simulator signal class members as top level signals in the test bench. Because writing portable testbenches is of a more complex subject, we stick with the static variant below:

In [4]:
# from myirl.test.ghdl import GHDL as Sim
from myirl.test.icarus import ICARUS as Sim
from cyrite.simulation import sim

@sim.testbench(Sim, time_unit = 'ns')
@block
def testbench():
    clk = Sim.ClkSignal()
    reset = Sim.ResetSignal(1, 1, isasync = False)
    
    assert reset.init == True
    
    ce = Sim.Signal(bool())
    
    led = Sim.Signal(bool())
    
    a = Sim.Signal(intbv()[10:])
    evt = Bool()

    uut = blinky(clk = clk, en = ce, reset = reset, outp = led, cfg = a)

    @always(delay(2))
    def clkgen():
        clk.next = ~ clk

    @sequence
    def main():
        evt.next = False
        yield delay(1)
        assert reset == True
        evt.next = ~evt
        yield delay(20)
        a.next = 6
        evt.next = ~evt
        yield delay(10)
        evt.next = ~evt
        reset.next = False
        yield delay(10)
        evt.next = ~evt
        assert led == False  # XXX Not on CXXRTL simulator
        ce.next = True
        yield delay(20)
        print("Wait for output to toggle")
        evt.next = ~evt
        # This waits for a rising edge of the led signal:
        yield led.posedge # XXX non-portable to CXXRTL simulator
        print("Output toggled")
        yield delay(50)
        evt.next = ~evt
        print("Simulation finished")
        raise StopSimulation

    return locals()

This test bench simply waits for the `led` output to become high using a `yield sig.posedge` construct.

A few important things to note:
   * A .posedge or .negedge event waiter using yield is not portable among all simulator back ends and is only guaranteed to work via a target language
   * Always terminate the simulation with a `raise StopSimulation` statement, otherwise simulation may keep running in some simulation back ends
   * The @block decorator is necessary when a translation of the testbench to another language occurs. This is not the case with a co-simulation setup, where the testbench is not translated to another HDL, like the CXXRTL simulator backend
   
Finally, we instance the testbench and run it:

In [5]:
def test_simulation(n):
    t = testbench()
    t.run(n, debug = True, wavetrace = "testbench.vcd")
    return t

t = test_simulation(200)

 Writing 'blinky' to file /tmp/blinky.v 
DEBUG Fallback wire for reset
 Writing 'testbench' to file /tmp/testbench.v 
 Note: Changing library path prefix to /tmp/ 
 Creating library file /tmp/module_defs.v 
DEBUG FILES ['/tmp/blinky.v', '/tmp/testbench.v']
==== COSIM stdout ====
VCD info: dumpfile testbench.vcd opened for output.
Wait for output to toggle 
Output toggled 
Simulation finished 
Stop Simulation



/home/cyrite/.local/lib/python3.10/site-packages/cyritehdl-0.1b0-py3.10-linux-x86_64.egg/myirl/test/icarus.py:40: UserWarning: Ignoring wavetrace argument for Verilog simulator
  warnings.warn("Ignoring wavetrace argument for Verilog simulator")
/home/cyrite/.local/lib/python3.10/site-packages/cyritehdl-0.1b0-py3.10-linux-x86_64.egg/myirl/kernel/_types.py:579: UserWarning: Non-ClkSignal 'led' edge event reference.
You might want to use ClkSignal instead.
  base.warnings.warn("Non-ClkSignal '%s'" % obj.identifier + \


## Waveform display

The waveform plugin for notebooks displays signals in a schematic way based on the changes of a sampling signal. Whenever this signal toggles, changes of all other signals are displayed. Unlike a true `.vcd` display such as GTKWave, this is not timing accurate.

In this case, we used a manually toggled `evt` signal to sample. Otherwise, we would end up with an unreadable trace if we used `clk`.

In [6]:
from cyrite import waveutils

waveutils.draw_wavetrace(t, "testbench.vcd", "evt")

In some cases we might want not to display all signals. This is done using a wave config specification which is a simple dictionary whose keys are the signal name and the value `None` or a delta time specification. The delta is only used in special cases where a signal change must be elaborated in particular.

In [7]:
wavecfg = { 'testbench.evt' : None,
           'testbench.reset' : None,
           'testbench.led' : None }

waveutils.draw_wavetrace(t, "testbench.vcd", "evt", cfg = wavecfg)

## Advanced simulation

The above check for a rising signal is a little cheap. We would like to verify a real blink timing.
Moreover, there are non-HDL simulation back ends that do not transpile the code as noted. We also get a warning for using the `led` signal as an event trigger. This is bad practise, because we actually don't know from the front end, when `led` is rising. Internally, the simulation will therefore check repeatedly for this signal to be rising and possibly burn a lot of time.

Then, the above simulation setup is specific to a HDL transpilation. We hoever might want to compile the synthesizeable elements into executable code and drive it from the test bench using co-simulation.
To easily switch between various simulation and synthesis targets, a top level design module class is the better investment into the future of possibly growing complexity of the design.

In particular, when designs get big, simulation can slow down. A way to speed this up is using a less timing precise but faster running edition of compiled logic using the CXXRTL backend of yosys.

Due to its architecture and lack of built-in delta stepping capabilities, a statement such as the above `yield led.posedge` is not allowed in a CXXRTL cosimulation context. It will have to be explicitely checked each clock cycle if it has changed.

See [Simulation](simulation.ipynb) on how complex simulation designs are set up.